# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

## Лабораторная работа 7

In [170]:
import pandas as pd
import numpy as np
import re
import nltk
import random
import pymorphy2
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_distances as cos_d

In [14]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Unzipping grammars/book_grammars.zip.
       | Downloading 


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [3]:
recipes = pd.read_csv('/content/ru_recipes_sample.csv')

In [4]:
recipes

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",Этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",Быстро и вкусно.
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","Сытный, овощной салатик, пальчики оближете."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",Картофельное пюре и куриные котлеты - вкусная ...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...","Вишневая наливка имеет яркий вишневый вкус, ко..."
...,...,...,...,...
3462,https://www.povarenok.ru/recipes/show/54574/,Мшош,"{'Чечевица': '1 стак.', 'Лук репчатый': '2 шт'...","Для тех, кто любит чечевицу... Вам сюда! Очень..."
3463,https://www.povarenok.ru/recipes/show/113494/,Мясные треугольники с баклажаном,"{'Фарш мясной': '400 г', 'Баклажан': '1 шт', '...",Баклажановые фантазии продолжаются! Предлагаю ...
3464,https://www.povarenok.ru/recipes/show/83228/,"""Болоньез"" по-новому","{'Фарш мясной': '400 г', 'Томаты в собственном...",Мое любимое блюдо лазанья. Но кушать только фа...
3465,https://www.povarenok.ru/recipes/show/172238/,Варенье из одуванчиков с апельсинами,"{'Цветки': '400 г', 'Сахар': '1300 г', 'Апельс...",Прошлым летом варила варенье из одуванчиков по...


In [11]:
new_desc = recipes.description.apply(lambda text: ''.join([i for i in re.findall('[а-яА-ЯёЁ0-9 ]+', text.lower())]))
recipes.description = new_desc

In [12]:
recipes.head()

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная к...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...


### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [25]:
w = list(set([i for index in recipes.description.apply(lambda descrip: nltk.word_tokenize(descrip)) for i in index]))
w

['грековой',
 'неё',
 'транспортировки',
 'пасту',
 'трубочки',
 'воспоминаниями',
 'поужинали',
 'хозяйство',
 'горчицей',
 'кальций',
 'витаминами',
 'потрясающее',
 'малиновый',
 'замочить',
 'причём',
 'запоминающееся',
 'которым',
 'решётке',
 'сэр',
 'тёлок',
 'тебе',
 'кунжутными',
 'италика',
 'сыыыром',
 'рыб',
 'классического',
 'айвы',
 'весеннему',
 'тетрадки',
 'вкуснейших',
 'дополним',
 'зимними',
 'основное',
 'эксперименты',
 'сайту',
 'победа',
 'тушенная',
 'кроме',
 'передаче',
 'события',
 'какао',
 'сушеные',
 'необычнонеповторимый',
 'тори',
 'пропустить',
 'кускус',
 'впитывают',
 'макаронных',
 'какбудто',
 'конкурсе',
 'необходим',
 'птицей',
 'морковная',
 'закусон',
 'жар',
 'физике',
 'плетеную',
 'зайка',
 'энное',
 'лиличке',
 'спрингроллов',
 'любопытно',
 'варила',
 'ржаного',
 'кукурузной',
 'омлетик',
 'намазывают',
 'прижимающиеся',
 'клубникой',
 'хранить',
 'бабушке',
 'праздничными',
 'перегружено',
 'сёмгипрелесть',
 'миядзаки',
 'лукомевропейска

In [29]:
for i in range(5):
  word_1 = random.choice(w)
  word_2 = random.choice(w)
  print(f"d({word_1}, {word_2}) = {nltk.edit_distance(word_1, word_2)}")

d(частых, замаринованная) = 13
d(потрачу, мраморный) = 8
d(французским, неожиданную) = 11
d(наваристые, макарончики) = 8
d(скромный, диету) = 8


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [30]:
def task_3(s1: str, s2: str) -> bool:
  words_1 = nltk.word_tokenize(s1)
  words_2 = nltk.word_tokenize(s2)
  P = 0
  L = max(len(words_1), len(words_2))
  for w1 in words_1:
    for w2 in words_2:
      d = nltk.edit_distance(w1, w2)
      if d < 2:
        P += 1
  
  return P/L > 0.8

In [32]:
recipes_id = recipes.url.apply(lambda url: int(url.split('/')[-2]))

In [36]:
task_3(recipes.loc[recipes_id == 135488].description.iloc[0], recipes.loc[recipes_id == 851934].description.iloc[0])

True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [53]:
df_task4 = pd.DataFrame(columns=['word', 'stemmed_word','normalized_word'])

In [54]:
df_task4.word = w

stemmed_words = nltk.SnowballStemmer('russian')
df_task4.stemmed_word = [stemmed_words.stem(word) for word in df_task4.word]

normalized_words = pymorphy2.MorphAnalyzer()
df_task4.normalized_word = [normalized_words.parse(word)[0].normalized.word for word in df_task4.word] 

In [55]:
df_task4 = df_task4.set_index('word')

In [56]:
df_task4

,stemmed_word,normalized_word
word,,
грековой,греков,греков
неё,не,она
транспортировки,транспортировк,транспортировка
пасту,паст,паста
трубочки,трубочк,трубочка
...,...,...
упаковках,упаковк,упаковка
духовкев,духовк,духовкева
настоящую,настоя,настоящий


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [161]:
stoplist = stopwords.words('russian')

In [60]:
description_no_stopwords = recipes.description.apply(lambda desc: " ".join([word for word in nltk.word_tokenize(desc) if word not in stoplist]))

In [61]:
recipes['description_no_stopwords'] = description_no_stopwords

In [64]:
recipes.head()

,url,name,ingredients,description,description_no_stopwords
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...,коктейль готовлю замороженной клубники клубник...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно,быстро вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете,сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная к...,картофельное пюре куриные котлеты вкусная клас...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...,вишневая наливка имеет яркий вишневый вкус кот...


In [79]:
all_words = [word for desc in recipes.description.apply(lambda descr: nltk.word_tokenize(descr)) for word in desc]
len(all_words)

102489

In [80]:
no_stopwords = [ word for desc in recipes.description_no_stopwords.apply(lambda descr: nltk.word_tokenize(descr)) for word in desc]
len(no_stopwords)

69449

In [94]:
fraction = (1-len(no_stopwords)/len(all_words))*100
print('Доля стоп-слов от всех: ', f'{fraction:.2f}%')

Доля стоп-слов от всех:  32.24%


In [83]:
fdist_before = FreqDist(all_words)
fdist_after = FreqDist(no_stopwords)

In [102]:
sorted(fdist_before.items(), key=lambda items: items[1], reverse=True)[:10]

[('и', 5043),
 ('в', 2567),
 ('с', 1932),
 ('на', 1642),
 ('очень', 1594),
 ('не', 1515),
 ('из', 1005),
 ('я', 972),
 ('а', 850),
 ('рецепт', 843)]

In [103]:
sorted(fdist_after.items(), key=lambda items: items[1], reverse=True)[:10]

[('очень', 1594),
 ('рецепт', 843),
 ('это', 728),
 ('блюдо', 521),
 ('вкусный', 459),
 ('просто', 434),
 ('вкусно', 366),
 ('приготовить', 342),
 ('вкус', 319),
 ('салат', 312)]

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [105]:
pancakes = recipes[recipes.name.apply(lambda name: "оладьи" in name.lower())]

In [106]:
pancakes_top5 = pancakes.sample(5)

In [107]:
vectorizer = TfidfVectorizer()

In [113]:
vectorizer_fit = vectorizer.fit_transform(pancakes_top5.description)

In [114]:
df_task6 = pd.DataFrame(vectorizer_fit.toarray(), columns=vectorizer.get_feature_names_out())

In [115]:
df_task6

,31,ароматные,без,блендер,более,весной,вкусная,вкусные,вкусных,вкусом,...,тают,тем,того,только,угощайтесь,упростился,фото,чашу,это,явным
0,0.000000,0.223607,0.223607,0.000000,0.000000,0.223607,0.000000,0.000000,0.223607,0.000000,...,0.000000,0.000000,0.000000,0.223607,0.223607,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.258607,0.000000,0.258607,0.000000,0.000000,0.000000,0.000000,0.000000,0.258607,0.000000
2,0.124306,0.000000,0.000000,0.124306,0.124306,0.000000,0.124306,0.000000,0.000000,0.000000,...,0.000000,0.124306,0.000000,0.000000,0.000000,0.124306,0.000000,0.124306,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.387757,0.000000,0.387757,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.387757
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.339992,0.000000,0.000000,0.000000


In [116]:
df_task6.iloc[:1]

,31,ароматные,без,блендер,более,весной,вкусная,вкусные,вкусных,вкусом,...,тают,тем,того,только,угощайтесь,упростился,фото,чашу,это,явным
0,0.0,0.223607,0.223607,0.0,0.0,0.223607,0.0,0.0,0.223607,0.0,...,0.0,0.0,0.0,0.223607,0.223607,0.0,0.0,0.0,0.0,0.0


In [117]:
pancakes_top5.description.iloc[1]

'нежнейшие оладушки которые тают во рту пришлось немного повозиться и подождать но это того стоит с наступающей масленицей'

7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [129]:
morph = pymorphy2.MorphAnalyzer()
w_regex = re.compile('^[а-яё]*$')

def vectorizer_n(string):
    return [morph.parse(word)[0].normalized.word for word in nltk.word_tokenize(string.lower()) if w_regex.search(word)]

In [162]:
cv = CountVectorizer(tokenizer=vectorizer_n, stop_words=stoplist)

In [166]:
recipes_cv = cv.fit_transform(pancakes_top5.description)

In [167]:
recipes_arr = recipes_cv.toarray()

In [168]:
recipes_arrn = recipes_arr / norm(recipes_arr, axis=1)[:, np.newaxis]

In [169]:
temp = recipes_arrn @ recipes_arrn.T
temp

array([[1.        , 0.07161149, 0.03615508, 0.09759001, 0.        ],
       [0.07161149, 1.        , 0.11651035, 0.10482848, 0.        ],
       [0.03615508, 0.11651035, 1.        , 0.10585122, 0.10585122],
       [0.09759001, 0.10482848, 0.10585122, 1.        , 0.        ],
       [0.        , 0.        , 0.10585122, 0.        , 1.        ]])

In [171]:
1 - cos_d(recipes_arr)

array([[1.        , 0.07161149, 0.03615508, 0.09759001, 0.        ],
       [0.07161149, 1.        , 0.11651035, 0.10482848, 0.        ],
       [0.03615508, 0.11651035, 1.        , 0.10585122, 0.10585122],
       [0.09759001, 0.10482848, 0.10585122, 1.        , 0.        ],
       [0.        , 0.        , 0.10585122, 0.        , 1.        ]])

Попробуем не через описание а через ингридиенты

In [172]:
cv = CountVectorizer(tokenizer=vectorizer_n, stop_words=stoplist)

In [173]:
recipes_cv = cv.fit_transform(pancakes_top5.ingredients)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn(


In [174]:
1 - cos_d(recipes_cv.toarray())

array([[1.        , 0.12964074, 0.77174363, 0.69310328, 0.66697297],
       [0.12964074, 1.        , 0.04724556, 0.        , 0.        ],
       [0.77174363, 0.04724556, 1.        , 0.79385662, 0.57452426],
       [0.69310328, 0.        , 0.79385662, 1.        , 0.40824829],
       [0.66697297, 0.        , 0.57452426, 0.40824829, 1.        ]])

In [180]:
for ingr in pancakes_top5.ingredients:
    print(ingr, '\n')

{'Яйцо куриное': '2 шт', 'Лук зеленый': '3-4 шт', 'Соль': 'по вкусу', 'Дрожжи': '0,5 ч. л.', 'Мука пшеничная': '220 г', 'Кефир': '300 мл', 'Масло растительное': '2 ст. л.'} 

{'Хлопья овсяные': '1 стак.', 'Молоко': '800 мл', 'Дрожжи': '1 пакет.', 'Сахар': '0.5 стак.', 'Соль': '1 щепот.', 'Мука пшеничная': '3 стак.'} 

{'Печень говяжья': '1000 г', 'Лук репчатый': '3 шт', 'Морковь': '2 шт', 'Крупа манная': '4 ст. л.', 'Сода': '0.75 ч. л.', 'Яйцо куриное': '4 шт', 'Сметана': '300 г', 'Сыр твердый': '100 г', 'Соль': None, 'Перец черный': None, 'Масло растительное': None, 'Вода': '100 мл'} 

{'Картофель': '3 шт', 'Семга': '200 г', 'Крахмал': '4 ч. л.', 'Соль': None, 'Перец черный': None, 'Масло растительное': None} 

{'Грудка куриная': None, 'Лук репчатый': '2 шт', 'Яйцо куриное': '4 шт', 'Крахмал': '3 ст. л.', 'Соль': 'по вкусу', 'Перец черный': 'по вкусу'} 



In [181]:
df_task7 = pd.DataFrame(temp, columns=list(pancakes_top5.name), index=list(pancakes_top5.name))
df_task7

,Оладьи с рубленым яйцом и луком,Оладьи овсяные дрожжевые,Оладьи из печени,Картофельные оладьи с семгой,Куриные оладьи
Оладьи с рубленым яйцом и луком,1.000000,0.071611,0.036155,0.097590,0.000000
Оладьи овсяные дрожжевые,0.071611,1.000000,0.116510,0.104828,0.000000
Оладьи из печени,0.036155,0.116510,1.000000,0.105851,0.105851
Картофельные оладьи с семгой,0.097590,0.104828,0.105851,1.000000,0.000000
Куриные оладьи,0.000000,0.000000,0.105851,0.000000,1.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [184]:
def task8(sim_df: pd.DataFrame) -> tuple:
    sim_df = sim_df.replace(1,0)
    recipe1 = sim_df.max().idxmax()
    recipe2 = sim_df.idxmax()[recipe1]
    return recipe1, recipe2

In [185]:
res = task8(df_task7)

In [186]:
df_task7.loc[res[0], res[1]]

1.0000000000000004

In [187]:
pancakes_top5.loc[pancakes_top5.name==res[0], 'description'].iloc[0]

'нежнейшие оладушки которые тают во рту пришлось немного повозиться и подождать но это того стоит с наступающей масленицей'

In [188]:
pancakes_top5.loc[pancakes_top5.name==res[1], 'description'].iloc[0]

'нежнейшие оладушки которые тают во рту пришлось немного повозиться и подождать но это того стоит с наступающей масленицей'